In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("pyspark")
sc = SparkContext(conf=conf)

In [2]:
btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6)])
btc = btc_raw.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))

In [3]:
#Version with broadcasting in the addEdge step on both Min_Selection_Step and Pruning_Step

def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge = NN_G_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    #Without broadcasting
    #addEdge = NN_G_u.join(v_min).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    H = addEdge.groupByKey().map(lambda x: (x[0], set(x[1])))#.filter(lambda x: len(x[1]) > 1)
    return H

def Pruning_Step(H):
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    v_min = H_filtered.map(lambda x: (x[0], min(x[1])))
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge2 = NN_H_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    #Without broadcasting
    #addEdge2 = NN_H_u.join(v_min).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    return G 

def cracker(G_i):
    count = 0
    while G_i.take(1):
        count += 1
        H_i = Min_Selection_Step(G_i)
        G_i = Pruning_Step(H_i)
        
    return count


In [5]:
#Version without broadcasting in the addEdge step on both Min_Selection_Step and Pruning_Step

def Min_Selection_Step2(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    addEdge = v_min.join(NN_G_u)
    H = addEdge.flatMap(lambda x: [(y, x[1][0]) for y in x[1][1]]).groupByKey().map(lambda x: (x[0], set(x[1])))
    return H

def Pruning_Step2(H):
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    v_min = H_filtered.map(lambda x: (x[0], min(x[1])))
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    addEdge2 = v_min.join(NN_H_u).flatMap(lambda x: [(x[1][0], y) for y in x[1][1]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    return G 

def cracker2(G_i):
    count = 0
    while G_i.take(1):
        count += 1
        H_i = Min_Selection_Step2(G_i)
        G_i = Pruning_Step2(H_i)
        
    return count


In [6]:
%%time
cracker(btc)

CPU times: user 145 ms, sys: 21.4 ms, total: 167 ms
Wall time: 2.68 s


3

In [7]:
%%time
cracker2(btc)

CPU times: user 300 ms, sys: 54.2 ms, total: 355 ms
Wall time: 8.57 s


3